In [1]:
#
# AI Chat Analyzer - Simplified for Learning (No RAG)
#
# Objective: A simple Gradio application that uses the OpenAI API to analyze
# a portion of an uploaded chat history file without a complex RAG pipeline.
#

# --- Step 1: Install Necessary Libraries ---
# We only need 'gradio' for the web UI and 'openai' for the API.
!pip install -q gradio openai

# --- Step 2: Import Required Modules ---
import gradio as gr
from openai import OpenAI
import os

# --- Step 3: Backend Logic (The "Brain" of the App) ---

def process_chat_request(user_question, chat_history, state_data):
    """
    This function handles the user's question and generates a response.

    Here's the simplified process (without RAG):
    1.  Slice Context: It takes the full chat history and "slices" it, using
        only the last N characters based on the "Context Window Size" slider.
        This slice becomes the context.
    2.  Construct Prompt: It creates a prompt for the AI, including the
        sliced context, and instructs the AI to answer based only on that.
    3.  Generate Answer: It sends the prompt to the OpenAI chat model
        (gpt-3.5-turbo) to get an answer.
    """
    # Retrieve the necessary data from the app's state.
    api_key = state_data.get("api_key")
    temp = state_data.get("temp")
    chat_content = state_data.get("chat_content")
    context_size = state_data.get("context_size")

    # Basic validation.
    if not all([api_key, temp is not None, chat_content, context_size]):
        raise gr.Error("Configuration is incomplete. Please restart.")
    if not user_question:
        raise gr.Error("Please enter a question.")

    # 1. Slice the chat content to get the context.
    # We take the last `context_size` characters from the file.
    # For example, if the size is 5000, we send the last 5000 characters.
    context_to_use = chat_content[-int(context_size):]

    # 2. Construct the prompt for the LLM.
    prompt = f"""
    You are a helpful assistant that analyzes a chat history. Based ONLY on the provided chat context below,
    answer the user's question. Do not use any external knowledge. If the answer is not in the context,
    state that you cannot find the answer in the provided chat history.

    --- CHAT CONTEXT ---
    {context_to_use}
    --- END OF CONTEXT ---

    QUESTION:
    "{user_question}"
    """

    try:
        # 3. Call the OpenAI API to generate the response.
        client = OpenAI(api_key=api_key)
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an expert chat analyst."},
                {"role": "user", "content": prompt}
            ],
            temperature=temp,
        )
        bot_response = completion.choices[0].message.content

    except Exception as e:
        raise gr.Error(f"An API error occurred: {e}")

    # Add the conversation turn to the chat history.
    chat_history.append((user_question, bot_response))
    return "", chat_history

# --- Step 4: Gradio User Interface (The "Look and Feel") ---

with gr.Blocks(theme=gr.themes.Soft(primary_hue="emerald", secondary_hue="emerald"), title="Chat Analyzer") as demo:
    # 'State' object stores data that persists across interactions.
    app_state = gr.State({})

    # --- Page 1: Welcome Screen ---
    with gr.Column(visible=True) as welcome_page:
        gr.Markdown(
            """
            <div style='text-align: center; font-family: "Garamond", serif; padding-top: 50px;'>
                <h1 style='font-size: 4em;'>Chat with your History</h1>
                <p style='font-size: 1.5em; color: #555;'>Analyze any chat export using OpenAI.</p>
            </div>
            """
        )
        gr.HTML(
            """
            <div style='text-align: center; padding: 30px;'>
                <img src='https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExd2Vjb3M2eGZzN2FkNWZpZzZ0bWl0c2JqZzZlMHVwZ2l4b2t0eXFpcyZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/YWjDA4k2n6d5Ew42zC/giphy.gif'
                     style='max-width: 400px; margin: auto; border-radius: 20px; box-shadow: 0 8px 16px rgba(0,0,0,0.1);' />
            </div>
            """
        )
        with gr.Row():
            enter_button = gr.Button("✨ Get Started ✨", variant="primary", scale=1)

    # --- Page 2: Configuration Screen ---
    with gr.Column(visible=False) as config_page:
        gr.Markdown("<h1 style='text-align: center;'>Setup Your Chat Environment</h1>")
        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 1. OpenAI API Key")
                api_key_input = gr.Textbox(label="API Key", type="password", placeholder="Enter your OpenAI key here...")
                gr.Markdown("### 2. Upload Chat File")
                chat_file_upload = gr.File(label="Upload Any .txt Chat Export")
            with gr.Column(scale=1):
                gr.Markdown("### 3. Customize AI Parameters")
                # This slider now controls how many characters from the end of the
                # file are used as context for the AI.
                context_slider = gr.Slider(
                    500, 20000, value=5000, step=500,
                    label="Context Window Size (Characters)"
                )
                temp_slider = gr.Slider(0.0, 1.0, value=0.5, step=0.1, label="Temperature (Creativity)")
        lets_chat_button = gr.Button("💬 Let's Chat! 💬", variant="primary")

    # --- Page 3: Chat Interface ---
    with gr.Column(visible=False) as chat_page:
        gr.Markdown("<h1 style='text-align: center;'>Chat Analyzer</h1>")
        chatbot_ui = gr.Chatbot(height=600, bubble_full_width=False, label="Chatbot")
        with gr.Row():
            user_input_box = gr.Textbox(placeholder="Ask a question about your chat...", scale=5, label="Your Question")
            submit_button = gr.Button("Send", variant="primary", scale=1)

    # --- Step 5: Event Handling (Connecting UI to Backend) ---

    def go_to_config():
        """Function to switch from the welcome page to the config page."""
        return {
            welcome_page: gr.Column(visible=False),
            config_page: gr.Column(visible=True)
        }

    def go_to_chat(api_key, chat_file, context_size, temp):
        """
        Function to save the config and switch to the chat page.
        """
        if not api_key:
            raise gr.Error("API Key is required.")
        if chat_file is None:
            raise gr.Error("A chat file must be uploaded.")

        # Read the content of the uploaded file.
        with open(chat_file.name, 'r', encoding='utf-8') as f:
            content = f.read()

        # Save the configuration into the app state.
        # Note: We are now storing the raw file content directly.
        new_state = {
            "api_key": api_key,
            "chat_content": content,
            "context_size": context_size,
            "temp": temp,
        }

        # Return the new state and the visibility updates for the pages.
        return (
            new_state,
            gr.Column(visible=False), # Hide config page
            gr.Column(visible=True)   # Show chat page
        )

    # This section "wires up" the buttons to their respective functions.
    enter_button.click(fn=go_to_config, inputs=None, outputs=[welcome_page, config_page])

    lets_chat_button.click(
        fn=go_to_chat,
        # The context_slider is now an input.
        inputs=[api_key_input, chat_file_upload, context_slider, temp_slider],
        outputs=[app_state, config_page, chat_page]
    )

    submit_button.click(
        fn=process_chat_request,
        inputs=[user_input_box, chatbot_ui, app_state],
        outputs=[user_input_box, chatbot_ui]
    )

    user_input_box.submit(
        fn=process_chat_request,
        inputs=[user_input_box, chatbot_ui, app_state],
        outputs=[user_input_box, chatbot_ui]
    )

# --- Step 6: Launch the App ---
print("\nLaunching Gradio App...")
demo.launch(debug=True, share=True)

/tmp/ipython-input-2972168350.py:133: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot_ui = gr.Chatbot(height=600, bubble_full_width=False, label="Chatbot")
/tmp/ipython-input-2972168350.py:133: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot_ui = gr.Chatbot(height=600, bubble_full_width=False, label="Chatbot")



Launching Gradio App...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://22e45ef1d599138162.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://22e45ef1d599138162.gradio.live
